In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.fft

from radon import get_operators

In [ ]:
data_path = "./data/"  

In [ ]:
device='cpu' #'cuda' for GPU, 'cpu' otherwise
radon_op, fbp_op = get_operators(n_angles=32, image_size=256, circle=True, device=device)

#forward model representing a radon transform
def A(x):
    y = radon_op(x)
    return y

In [ ]:
plt.gray()
img = torch.load(data_path+'0')
plt.imshow(img[0])

In [ ]:
sinogram = A(img.unsqueeze(1))  # unsqueeze because A expects a channel dimension at location 1
print(sinogram.shape)
plt.imshow(sinogram.squeeze())

In [ ]:
#We use filtered back projection as an estimate of A.T
def AT(y):
    x=fbp_op(y)
    x = x.to(torch.float32)
    return x

reconstruction = AT(sinogram)
plt.imshow(reconstruction.squeeze())

In [ ]:
# torch dataset that returns sinograms and ground-truth images
class CTDataSet(torch.utils.data.Dataset):
    
    def __init__(self, path):
        self.path = path

    def __len__(self):
        return len(os.listdir(self.path))
    
    def __getitem__(self, idx):
        img = torch.load(self.path+str(idx)).to(device)
        sino = A(img.unsqueeze(1))
        return img.squeeze(), sino.squeeze()

In [ ]:
data_set = CTDataSet(data_path)

#Split dataset into 99 imgs for training and 1 img for validation
train_set, val_set = torch.utils.data.random_split(data_set,[99,1],generator=torch.Generator().manual_seed(0))

#Dataloader
train_dl = DataLoader(train_set, batch_size=1)
val_dl = DataLoader(val_set, batch_size=1)

In [ ]:
# fill in code here
# you are free to create as many classes as you need



#Whole Variational Network    
class VarNet(nn.Module):
    def __init__(self, num_cascades=5):
        super(VarNet, self).__init__()
        
        # fill in code here
        
    def forward(self, y):
        
        # fill in code here
         
        return

### Below are funtions to train and test the network

You can modify the functions or the hyperparameters for training as you wish

In [ ]:
##Loss
def mse(gt: torch.Tensor, pred:torch.Tensor)-> torch.Tensor:
    loss = torch.nn.MSELoss()
    return loss(gt,pred)

#train function
def train_step(model, optimizer, dataloader_sample):
    model.train()
    
    # reset optimizer's gradient
    optimizer.zero_grad()

    # define variables
    x, y = dataloader_sample
      
    # get the prediction
    pred = model(y.unsqueeze(1))[-1].squeeze(1)
    pred_loss = mse(pred, x)
    
    #one step of training
    pred_loss.backward()
    optimizer.step()

    return pred_loss.item()

#test function
def validation_step(model, dataloader_sample): 
    model.eval()
    
    with torch.no_grad(): 
        
        x, y = dataloader_sample
        
        # get the prediction
        pred = model(y.unsqueeze(1))[-1].squeeze(1)
        pred_loss = mse(pred, x)

    return pred_loss.item()

### Training

As optimizer, we choose the Adam optimizer (a standard adaptive gradient method). We then train the model for 10 epochs; training for more epochs gives better results, but after 10 epochs we already get a model that works reasonably well for image recovery.

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.0005, weight_decay=0)

max_epoch = 10
mse_train=[]
mse_val= []

for epoch in tqdm(range(max_epoch)):
    # Initialize Loss and Accuracy
    train_loss = val_loss= 0.0
    
    ### Training Phase
    
    ## Iterate over the train_dataloader
    with tqdm(total=len(train_dl)) as pbar:
        for sample in train_dl:            
            curr_loss = train_step(model, optimizer, sample)
            train_loss += curr_loss / len(train_dl) 
            pbar.update(1)
    
    mse_train.append(train_loss)


    ### Validation Phase 
    
    ## Validation_dataloader
    with tqdm(total=len(val_dl)) as pbar:
        for sample in val_dl: 
            curr_loss = validation_step(model, sample)
            val_loss += curr_loss / len(val_dl)
            pbar.update(1)
    
    mse_val.append(val_loss) 
    
    print(epoch, train_loss, val_loss)    


In [ ]:
plt.plot(mse_val)
plt.title('Validation Error')
plt.xlabel('epoch')
plt.ylabel('MSE')

### Visualization how VarNet reconstructs an image

In [ ]:
sample=next(iter(val_dl))  #img from val set

#reconstruction
with torch.no_grad(): 
    x, y = sample
    pred = model(y.unsqueeze(1))
    fbp = AT(y.unsqueeze(1))

plt.gray()
fig, ax = plt.subplots(1, 7,figsize=(15, 15))

ax[0].imshow(fbp.squeeze().cpu())
ax[0].set_xticks([])
ax[0].set_yticks([])
ax[0].set_title('FBP') 

for i in range(5):
    ax[i+1].imshow(pred[i].squeeze().cpu())
    ax[i+1].set_xticks([])
    ax[i+1].set_yticks([])
    ax[i+1].set_title('Cascade '+ str(i+1))
    
ax[6].imshow(x.squeeze().cpu())   
ax[6].set_xticks([])
ax[6].set_yticks([])
ax[6].set_title('Ground Truth')  